In [1]:
# Daniel loads his data
from load_DER_data import load_dataset
raw_data = load_dataset()

fetching NWP...
Simulations start at:  2016-01-01 18:00:00
Sample time of Sims is:  0:05:00
Sample time of simulation: 0:05:00
missing  503  samples from  2016-12-31 00:00:00 to  2017-01-01 18:00:00
Setting flags for those datapoints to 1
(649, 12) (503, 13)
missing  1079  samples from  2017-03-23 00:00:00 to  2017-03-26 18:00:00
Setting flags for those datapoints to 1
(649, 12) (1079, 13)
replacing the fauly values with the mean of the array
fetched NWP ... now fetching PV Data
Loading generation data from  2016-01-0117:00:00  to  2017-12-3117:05:00
fetched PV ... now concatenating


In [2]:
from datasets_utils import datasets_from_data
inp, ev, pdf = datasets_from_data(raw_data,
                                  sw_len_samples=int(7*24*(60/5)),
                                  fc_len_samples=int(1*24*(60/5)),
                                  fc_steps=48,
                                  fc_tiles=33,
                                  target_dims=[0,1,2,3,4],
                                  plot=True,
                                 steps_to_new_sample=15)

c:\users\danielcmaz\appdata\local\programs\python\python36\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)
c:\users\danielcmaz\appdata\local\programs\python\python36\lib\site-packages\sklearn\preprocessing\data.py:164: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "


<Figure size 2000x1500 with 17 Axes>

0 percent converted
9 percent converted
19 percent converted
29 percent converted
39 percent converted
49 percent converted
59 percent converted
69 percent converted
79 percent converted
89 percent converted


In [11]:
sw_train = sw_inputs[:int(0.8*sw_inputs.shape[0]),:,:]
sw_test = sw_inputs[int(0.8*sw_inputs.shape[0]):,:,:]
pdf_train = pdf_targets[:int(0.8*sw_inputs.shape[0]),:,:]
pdf_test = pdf_targets[int(0.8*sw_inputs.shape[0]):,:,:]
ev_train = ev_targets[:int(0.8*sw_inputs.shape[0]),:]
ev_test = ev_targets[int(0.8*sw_inputs.shape[0]):,:]

print(sw_train.shape, 'to ', sw_test.shape)
print(pdf_train.shape, 'to ', pdf_test.shape)
print(ev_train.shape, 'to ', ev_test.shape)


(10705, 2016, 17) to  (2677, 2016, 17)
(10705, 48, 33) to  (2677, 48, 33)
(10705, 48) to  (2677, 48)


In [25]:
class proper_FFW(tf.keras.layers.Layer): 
    def __init__(self, units, dropout_rate=0.,
                 initializer=tf.initializers.he_uniform(),
                 bias_init=tf.initializers.zeros(),
                 w_reg=[1e-6, 1e-6], concat_input=False, residual_connect=False):
        super(proper_FFW, self, ).__init__()
        
        self.concat_input = concat_input
        self.residual_connect = residual_connect
        
        self.FFW = keras.layers.Dense(units=units,
                                     kernel_initializer=initializer,
                                     bias_initializer=bias_init,
                                     kernel_regularizer=keras.regularizers.l1_l2(l1=w_reg[0], l2=w_reg[1]))
        self.drop = keras.layers.Dropout(rate=dropout_rate)
        self.norm = keras.layers.experimental.LayerNormalization()
        self.activation = keras.layers.ELU()
        
    
    def call(self,data_in):
        data_out = self.FFW(data_in)
        data_out = self.norm(data_out)
        data_out = self.activation(data_out)
        data_out = self.drop(data_out)
        
        return data_out

In [76]:
class proper_GRU(tf.keras.layers.Layer): 
    def __init__(self, units, dropout_rate=0.2,
                 initializer=tf.initializers.he_normal(),
                 bias_init=tf.initializers.zeros(),
                 w_reg=[1e-6, 1e-6], concat_input=False, residual_connect=False):
        super(proper_GRU, self, ).__init__()
        
        self.concat_input = concat_input
        self.residual_connect = residual_connect
        
        self.GRU = keras.layers.GRU(units=units,
                                    activation='tanh',
                                     kernel_initializer=initializer,
                                     bias_initializer=bias_init,
                                     kernel_regularizer=keras.regularizers.l1_l2(l1=w_reg[0], l2=w_reg[1]),
                                    return_sequences=True)
        self.drop = keras.layers.Dropout(rate=dropout_rate,)
        self.norm = keras.layers.experimental.LayerNormalization()
        self.activation = keras.layers.ELU()
        
    
    def call(self,data_in):
        data_out = self.GRU(data_in)
        data_out = self.norm(data_out)
        data_out = self.activation(data_out)
        data_out = self.drop(data_out)
        
        return data_out

In [36]:
# build the FF model
import tensorflow as tf
from tensorflow import keras
keras.backend.clear_session()
model =  keras.Sequential([
    keras.layers.InputLayer(sw_inputs[0].shape),
    keras.layers.Flatten(),
    proper_FFW(units=4*1024),
    proper_FFW(units=2*1024),
    proper_FFW(units=1024),
    proper_FFW(units=512),
    proper_FFW(units=128),
    proper_FFW(units=ev_targets[0].shape[0], dropout_rate=0.0)
  ])

optimizer = tf.keras.optimizers.Adam(1e-4)

model.compile(loss=tf.losses.MSE,
            optimizer=optimizer,
            metrics=[tf.losses.MAE, tf.losses.MSE])

model.summary()


model.fit(x=sw_train, y=ev_train,
          batch_size=256, epochs=15,
          validation_split = 0.15)

model.evaluate(sw_test, ev_test, verbose=0)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 34272)             0         
_________________________________________________________________
proper_ffw (proper_FFW)      (None, 4096)              140390400 
_________________________________________________________________
proper_ffw_1 (proper_FFW)    (None, 2048)              8394752   
_________________________________________________________________
proper_ffw_2 (proper_FFW)    (None, 1024)              2100224   
_________________________________________________________________
proper_ffw_3 (proper_FFW)    (None, 512)               525824    
_________________________________________________________________
proper_ffw_4 (proper_FFW)    (None, 128)               65920     
_________________________________________________________________
proper_ffw_5 (proper_FFW)    (None, 48)                6

In [79]:
# build the FF model
import tensorflow as tf
from tensorflow import keras
keras.backend.clear_session()
w_reg=[1e-6, 1e-6]
initializer=tf.initializers.he_normal()
bias_init=tf.initializers.zeros()


model = tf.keras.models.Sequential()
model.add(keras.layers.InputLayer(sw_inputs[0].shape))
model.add(keras.layers.Reshape([ev_targets[0].shape[0],int(sw_inputs[0].shape[1]*(sw_inputs[0].shape[0]/ev_targets[0].shape[0]))]))
model.add(proper_GRU(units=1024))
model.add(proper_GRU(units=1024))
model.add(proper_GRU(units=1024))
model.add(proper_GRU(units=512))
model.add(proper_GRU(units=512))
model.add(proper_FFW(units=248, dropout_rate=0.3))
model.add(proper_FFW(units=1, dropout_rate=0.0))
model.add(keras.layers.Reshape([48]) )

optimizer = tf.keras.optimizers.Adam(1e-4)

model.compile(loss=tf.losses.MSE,
            optimizer=optimizer,
            metrics=[tf.losses.MAPE, tf.losses.MSE])

model.summary()

model.fit(x=sw_train, y=ev_train,
          batch_size=256, epochs=15,
          validation_split = 0.15)

model.evaluate(sw_test, ev_test, verbose=0)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape (Reshape)            (None, 48, 714)           0         
_________________________________________________________________
proper_gru (proper_GRU)      (None, 48, 1024)          5347328   
_________________________________________________________________
proper_gru_1 (proper_GRU)    (None, 48, 1024)          6299648   
_________________________________________________________________
proper_gru_2 (proper_GRU)    (None, 48, 1024)          6299648   
_________________________________________________________________
proper_gru_3 (proper_GRU)    (None, 48, 512)           2363392   
_________________________________________________________________
proper_gru_4 (proper_GRU)    (None, 48, 512)           1576960   
_________________________________________________________________
proper_ffw (proper_FFW)      (None, 48, 248)           1

[0.9542812668747274, 339.40506, 0.5269157]

In [ ]:
class Encoder(tf.keras.Model):
    def __init__(self, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.gru = tf.keras.layers.GRU(self.enc_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')

    def call(self, x, hidden):
        output, state = self.gru(x, initial_state = hidden)
        return output, state

    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.enc_units))

In [31]:
class BahdanauAttention(tf.keras.Model):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, query, values):
        # hidden shape == (batch_size, hidden size)
        # hidden_with_time_axis shape == (batch_size, 1, hidden size)
        # we are doing this to perform addition to calculate the score
        hidden_with_time_axis = tf.expand_dims(query, 1)

        # score shape == (batch_size, max_length, hidden_size)
        score = self.V(tf.nn.tanh(
            self.W1(values) + self.W2(hidden_with_time_axis)))

        # attention_weights shape == (batch_size, max_length, 1)
        # we get 1 at the last axis because we are applying score to self.V
        attention_weights = tf.nn.softmax(score, axis=1)

        # context_vector shape after sum == (batch_size, hidden_size)
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, attention_weights

In [32]:
class Decoder(tf.keras.Model):
    def __init__(self, dec_units, batch_sz):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.gru = tf.keras.layers.GRU(self.dec_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')
        self.fc = tf.keras.layers.Dense(vocab_size)

        # used for attention
        self.attention = BahdanauAttention(self.dec_units)

    def call(self, x, hidden, enc_output):
        # enc_output shape == (batch_size, max_length, hidden_size)
        context_vector, attention_weights = self.attention(hidden, enc_output)

        # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

        # passing the concatenated vector to the GRU
        output, state = self.gru(x)

        # output shape == (batch_size * 1, hidden_size)
        output = tf.reshape(output, (-1, output.shape[2]))

        # output shape == (batch_size, vocab)
        x = self.fc(output)

        return x, state, attention_weights

In [ ]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)

In [33]:
@tf.function
def train_step(inp, targ, enc_hidden):
    loss = 0

    with tf.GradientTape() as tape:
        enc_output, enc_hidden = encoder(inp, enc_hidden)

        dec_hidden = enc_hidden

        dec_input = tf.expand_dims([targ_lang.word_index['<start>']] * BATCH_SIZE, 1)

        # Teacher forcing - feeding the target as the next input
        for t in range(1, targ.shape[1]):
            # passing enc_output to the decoder
            predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

            loss += loss_function(targ[:, t], predictions)

            # using teacher forcing
            dec_input = tf.expand_dims(targ[:, t], 1)

    batch_loss = (loss / int(targ.shape[1]))

    variables = encoder.trainable_variables + decoder.trainable_variables

    gradients = tape.gradient(loss, variables)

    optimizer.apply_gradients(zip(gradients, variables))

    return batch_loss

In [35]:
EPOCHS = 10
import time

for epoch in range(EPOCHS):
    start = time.time()

    enc_hidden = encoder.initialize_hidden_state()
    total_loss = 0

    for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
        batch_loss = train_step(inp, targ, enc_hidden)
        total_loss += batch_loss

        if batch % 100 == 0:
            print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                         batch,
                                                         batch_loss.numpy()))
    # saving (checkpoint) the model every 2 epochs
    if (epoch + 1) % 2 == 0:
        checkpoint.save(file_prefix = checkpoint_prefix)

    print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      total_loss / steps_per_epoch))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

NameError: name 'encoder' is not defined